# Импорт необходимых библиотек

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Загрузка и предобработка данных

Считаем токсичным любой комментарий, где target > 0.7.

In [3]:
np.random.seed(0)

df = pd.read_csv("data.csv")
texts = df["comment_text"]
labels = (df["target"] > 0.7).astype(int)

# Задание 1
Используем 70% данных для обучения и 30% — для теста.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.3, random_state=42
)

# Задание 2

Используем CountVectorizer

In [5]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Задание 3

Обучим модель, сделаем предсказания и оценим точность.


In [6]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.9273


# Задание 4

Опишем функцию, которая возвращает вероятность токсичности от 0 до 1.


In [7]:
def predict_toxicity(text):
    vectorized = vectorizer.transform([text])
    prob = model.predict_proba(vectorized)[0][1]
    return prob

# Задание 5

Протестируем модель на базовых примерах

In [11]:
print("‘Apples are stupid’:", predict_toxicity("Apples are stupid"))
print("‘I love apples’:", predict_toxicity("I love apples"))

‘Apples are stupid’: 0.9991497306309441
‘I love apples’: 0.05671043399691718


# Задание 6

Извлекаем самые токсичные слова по весу коэффициентов (топ-10)


In [12]:
feature_names = vectorizer.get_feature_names_out()
weights = model.coef_[0]
top_idx = np.argsort(weights)[-10:][::-1]

print("Топ-10 токсичных слов:")
for i in top_idx:
    print(f"{feature_names[i]}: {weights[i]:.4f}")


Топ-10 токсичных слов:
stupid: 9.2929
idiot: 8.8017
idiots: 8.5161
stupidity: 7.5930
idiotic: 6.8486
crap: 6.5977
dumb: 6.4676
hypocrite: 6.4396
pathetic: 6.4346
moron: 6.4076


# Задание 7

Проверим, как модель реагирует на слово "pathetic" в другом контексте


In [15]:
sample = "Don’t you dare call Hagrid pathetic" 
print(predict_toxicity(sample))

0.9926929910253336


(Фраза из книги Джоан Роулинг Гарри Поттер и узник Азкабана)

Теперь попробуем подобрать фразу, где слово будет использовано в другом значении - пусть, в качестве примера будет фраза из книги Максима Горького "В людях", где слово pathetic используется в значении "печальный"

In [20]:
sample = " I read it through in one night, and, surprised at the new experience, read the simple, pathetic story over again."
print(predict_toxicity(sample))

0.8727439256360728


Фраза по мнению модели менее токсична.

# Задание 8:

Проверка на предвзятость модели по этническим и религиозным фразам

In [21]:
bias_samples = [
    "I have a christian friend",
    "I have a muslim friend",
    "I have a white friend",
    "I have a black friend"
]

for text in bias_samples:
    print(f"‘{text}’: {predict_toxicity(text):.4f}")

‘I have a christian friend’: 0.1881
‘I have a muslim friend’: 0.5027
‘I have a white friend’: 0.3928
‘I have a black friend’: 0.5798


У модели явно прослеживается предвзятость. Bias не этичен.

# Задание 9

Если модель обучалась на данных, где определённые группы чаще встречались в токсичном контексте, она будет предвзята.

Это называется **historical bias** (историческая предвзятость).


# Задание 10

Как улучшить этичность модели?

1. Применить регуляризацию и фильтрацию слов с культурным bias
2. Использовать специальные fairness-aware модели или postprocessing